In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}, {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}, {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                options = dropdown_options,
                                value = 'ALL',
                                placeholder = 'Select Launch Site',
                                searchable = True
                                ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
    removed_sites = spacex_df[spacex_df['Launch Site'] == entered_site]
    single_site = removed_sites.groupby('class').count().reset_index()
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class',
        names = 'Launch Site',
        title = 'Total Successful Launches By Site')
        return fig
    else:
        fig = px.pie(single_site, values='Launch Site',
        names = single_site['class'].map({1: 'Success', 0: 'Failure'}),
        title = 'Percentage Successful Launches At Site')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, entered_mass):
    site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    filtered_df = spacex_df[(entered_mass[0] < spacex_df['Payload Mass (kg)']) & (spacex_df['Payload Mass (kg)'] < entered_mass[1])]
    single_site_df = site_df[(entered_mass[0] < site_df['Payload Mass (kg)']) & (site_df['Payload Mass (kg)'] < entered_mass[1])]

    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x=filtered_df['Payload Mass (kg)'], y=filtered_df['class'], color = 'Booster Version Category')
        return fig

    else:
        fig = px.scatter(single_site_df, x=single_site_df['Payload Mass (kg)'], y=single_site_df['class'], color = 'Booster Version Category')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
